# Checkpoint 3

**Due: Friday, 2 December, 2022 at 5:00pm GMT**

Total points: 100

### Read This First
1. Use the constants provided in the cells. Do not use your own constants.

2. Wherever you see `raise NotImplementedError()`, remove that line and put your code there.

3. Put the code that produces the output for a given task in the cell indicated. You are welcome to add as many cells as you like for imports, function definitions, variables, etc. Do not alter the argument list of functions that are given to you.

4. Your notebook must run correctly when executed once from start to finish. Your notebook will be graded based on how it runs, not how it looks when you submit it. To test this, go to the *Kernel* menu and select *Restart & Run All*.

5. Once you are happy with it, clear the output by selecting *Restart & Clear Output* from the *Kernel* menu.

6. Submit through Noteable.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import time

In [ ]:
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 14

## Problem 1: the flight of batted baseballs (50 pts)

Batted baseballs experience enough air resistance to alter their paths from a simple parabolic motion. To properly model the flight of a baseball, we must consider the drag force, which is defined as

$
\begin{align}
\large
F_{D} = -\frac{1}{2} C_{D} A \rho v^{2},
\end{align}
$

where $C_{D}$ is the drag coefficient, $A$ is the cross-sectional area of the ball, $\rho$ is the density of air, and $v$ is the velocity of the ball relative to the air. The drag force is oriented opposite to the velocity of the ball.

The system of equations describing the motion of an object experiencing the forces of gravity and air resistance is given by

$
\begin{align}
\large
\frac{dx}{dt} = v_{x}
\end{align}
$

$
\begin{align}
\large
\frac{dy}{dt} = v_{y}
\end{align}
$

$
\begin{align}
\large
\frac{dv_{x}}{dt} = \frac{F_{D_x}}{m}
\end{align}
$

$
\begin{align}
\large
\frac{dv_{y}}{dt} = \frac{F_{D_y}}{m} - g
\end{align}
$

where $F_{D_x}$ and $F_{D_y}$ are the $x$ and $y$ components of the drag force and $m$ is the mass of the ball.

The cell below defines a function describing simple projectile motion without air resistance.

In [ ]:
g = 9.80665 # m/s^2
def projectile_motion(t, f):
    """
    f0 = x  => dx/dt  = vx
    f1 = y  => dy/dt  = vy
    f2 = vx => dvx/dt = 0
    f3 = vy => dvy/dt = - g
    """
    
    vals = np.zeros_like(f)
    vals[0] = f[2]
    vals[1] = f[3]
    vals[2] = 0
    vals[3] = - g

    return vals

## Task 1: 20 pts

Compute the motion of a batted ball **experiencing air resistance** under the following conditions.

1. The initial position of the ball is x = 0 m and y = 1 m (the approximate height of a hittable pitch).
2. The initial velocity of the ball is 50 m/s at an angle of 42 degrees with respect to the ground.
3. The ball will land in the seating area beyond the field of play. Nearest to the field, the seats are at a height of 3.5 m up from the ground. The ball should be considered to have landed when it reaches this height (3.5 m).

To do this you must define a new system of equations describing the motion of the ball under gravity and air resistance. You may start with the `projectile_motion` defined function above if you like. The relevant constants are provided in the cell below. Use these in your calculation.

In the cell with `task1`, write a function that returns the time of flight in seconds and the final x displacement of the baseball in meters when it lands in the stands. This function should accept two arguments: the initial speed of the baseball in m/s and the initial angle in degrees. **Do not modify the existing argument list for the `task1` function.** In a subsequent cell, we will call your function with the initial conditions given above. Refer to the testing cell to see how the function will be called.

Your answers must be within 0.1 s and 0.1 m of the correct time and displacement, which are not given.

In [ ]:
# baseballs
m = 0.145 # mass in kg
c = 23.2  # circumference in cm
r = c / 2 / np.pi
A = np.pi * (r)**2 / 10000 # m^2
Cd = 0.34

# Earth-related constants
rhoE = 1.18 # air density at sea level kg/m^3
g = 9.80665 # m/s^2

In [ ]:
from scipy.integrate import solve_ivp

In [ ]:
def projectile_motion_drag(t, f):
    """
    Similiar function to one above but with added drag
    
    f0 = x  => dx/dt  = vx
    f1 = y  => dy/dt  = vy
    f2 = vx => dvx/dt = Fx/m
    f3 = vy => dvy/dt = Fy/m - g
    """
    #setting up drag forces
    v = np.sqrt(f[2]**2 + f[3]**2)
    Fx = -1/2 * Cd * A *rhoE * f[2]*v
    Fy = -1/2 * Cd * A *rhoE * f[3]*v
    
    vals = np.zeros_like(f)
    vals[0] = f[2]
    vals[1] = f[3]
    vals[2] = Fx/m
    vals[3] = Fy/m - g

    return vals

#define event to stop when the balls reaches 3.5 meters while moving in the negative direction
def y_distance(t,f):
    return f[1] - 3.5
y_distance.terminal = True
y_distance.direction = -1

In [ ]:
def task1(vi, theta):
    #trigonometry to access x and y components (convert to radians)
    vx = vi * np.cos(np.radians(theta))
    vy = vi * np.sin(np.radians(theta))
    
    #create arrays for inital conditions
    fi = np.array([0,1,vx,vy])
    tvalues = np.linspace(0,10,1000)
    
    #solve using scipy integrate solve_ivp
    sol = solve_ivp(projectile_motion_drag, (0, 10), fi,t_eval = tvalues,events=(y_distance), dense_output=True)

    return sol.t.max(), sol.y[0].max()

In [ ]:
vi = 50 # m/s
theta = 42 # degrees

tfinal1, xfinal1 = task1(vi, theta)
print (f"Flight time: {tfinal1} s.")
print (f"Final x displacement: {xfinal1} m.")

# We will test against the correct answer.


## Task 2: 20 pts

Can you hit a ball that will land in the same spot in half the time?

In the cell below with `task2`, write a function that computes the initial speed and angle required for a ball to land in the same spot as in Task 1. This function should accept a single argument, the desired flight time in seconds. **Do not modify the argument list of `task2`.** The function should return the initial speed in m/s and initial angle in degrees. Refer to the testing cell below to see how the function will be called and tested. Your answer should result in a flight time that is half of the flight time from task 1 to within 1%.

In [ ]:
from scipy.integrate import solve_bvp

In [ ]:
def projectile_motion_drag2(x, y):
    """
    Used same function as above but changed it so can be used with solve_bvp
    
    f0 = x  => dx/dt  = vx
    f1 = y  => dy/dt  = vy
    f2 = vx => dvx/dt = Fx/m
    f3 = vy => dvy/dt = Fy/m - g
    """
    v = np.sqrt(y[2]**2 + y[3]**2)
    Fx = -1/2 * Cd * A *rhoE * y[2]*v
    Fy = -1/2 * Cd * A *rhoE * y[3]*v
    
    vals = np.zeros_like(y)
    a = y[2]
    b = y[3]
    c = Fx/m
    d = Fy/m - g

    return np.vstack((a,b,c,d))

#define the boundary conditions
def bc(ya,yb):
    return np.array([ya[0], ya[1]-1,yb[1]-3.5,yb[0] -xfinal1])

In [ ]:
def task2(tflight):
    x = np.linspace(0, tflight, 100)
    y = np.zeros((4, x.size))
    
    #setting initial guess
    y[1] = 1
    y[2] = 60
    y[3] = 60
    
    sol = solve_bvp(projectile_motion_drag2, bc, x, y)
    
    #final answer using trig, converting back to degrees in return statement
    v_i = np.sqrt(sol.y[2][0]**2 + sol.y[3][0]**2)
    theta_i = np.arctan2(sol.y[3][0],sol.y[2][0])

    return v_i,np.degrees(theta_i)

In [ ]:
factor = 2
my_vi, my_angle = task2(tfinal1 / factor)
print (f"Initial velocity: {my_vi} m/s, angle: {my_angle} degrees.")

tfinal2, xfinal2 = task1(my_vi, my_angle)
print (f"Flight time: {tfinal2} s.")
print (f"Final x displacement: {xfinal2} m.")

print (f"Task1 flight time: {tfinal1} s, task2 flight time: {tfinal2} s, ratio: {tfinal1/tfinal2}")
diff = np.abs(factor - tfinal1/tfinal2)
print (f"{diff=}")
assert diff < 0.01 * factor

## Task 3: 10 pts

Make an animation showing the flight of both baseballs (i.e., x displacement on x axis and y displacement on y axis). Time it so the slower ball launches first, followed by the faster ball, and they land at the same time.

The axes function `ax.set_aspect('equal')` (for a given axes object called `ax`) can be used to make distances the same on the x and y axes.

In [ ]:
def task3(vi, theta):
    #Decided to use a new function for task 3 to create full arrays for each ball
    #Arrays were filled with positions and velocity for animation
    #Could of done animation on the fly for better performance but this method still works well enough

    
    vx = vi * np.cos(np.radians(theta))
    vy = vi * np.sin(np.radians(theta))
    
    fi = np.array([0,1,vx,vy])
    tvalues = np.linspace(0,10,1000)
    
    sol = solve_ivp(projectile_motion_drag, (0, 10), fi,t_eval = tvalues,events=(y_distance), dense_output=True)

    return np.array((sol.y[0],sol.y[1]))

In [ ]:
from matplotlib import animation
from IPython.display import HTML

#Hard coded but could be implemented generally
array1 = task3(50,42)
array = task3(my_vi,my_angle)

N = len(array1[0]) - len(array[0])

#Made first elements for faster ball 0 so it starts when slower ball is halfway
a = np.concatenate((np.zeros(N),array[0]),axis = 0) 
b = np.concatenate((np.zeros(N),array[1]),axis = 0)
array2 = np.array((a,b))

#have to set initial conditions
array2[1][0] = 1

#plot statements
fig, ax = plt.subplots()
ax.set_ylim(0,50)
ax.set_xlim(0,130)
ax.set_xlabel("X displacement (m)")
ax.set_ylabel("Y displacement (m)")
ax.set_aspect("equal")
line, = ax.plot([], [], lw=2)

patch1 = plt.Circle((5, -5), 1.5, fc='y')
patch2 = plt.Circle((5, -5), 1.5, fc='b')

#create array of patches
patches = []
patches.append(patch1)
patches.append(patch2)

#initalise patches in patch array
def init():
    for i in patches:
        i.center = (0, 1)
        ax.add_patch(i)
    return patches

#using arrays, animate the balls
def animate(i):
    patches[0].center = array1[0][i],array1[1][i]
    patches[1].center = array2[0][i],array2[1][i]
    return patches

anim = animation.FuncAnimation(fig, animate, 
                               init_func=init, 
                               frames=540, 
                               interval=20,
                               blit=True)

HTML(anim.to_jshtml())

## Problem 2: harmonics of the square wave (25 pts)

A square wave is composed of a fundamental tone (at wavenumber $\omega=1$) and a series of harmonics at odd wavenumbers. The amplitudes of the harmonics obey the following relation:

$
\begin{align}
\large
\frac{|c_k(\omega=3,5,7,...)|}{|c_k(\omega=1)|} \propto \omega^{\alpha},
\end{align}
$

where $c_k(\omega)$ are the Fourier coefficients as a function of wavenumber of the square wave and $\alpha$ is a constant. $c_k(\omega=1)$ is the Fourier coefficient of the fundamental tone, i.e., at $\omega=1$.

In the cell below, you are given a square wave signal.

In [ ]:
from scipy import signal

tsqu = np.linspace(0, 20*np.pi, 1001)
ysqu = signal.square(tsqu)

plt.plot(tsqu, ysqu)
plt.xlabel('t')
plt.ylabel('f(t)')
plt.title('square wave')
plt.show()

## Task 1: 20 pts

In the cell below, use the square wave signal define above to calculate the value of $\alpha$ from the previous expression. Do this by taking the FFT of the signal, locating the peaks at the harmonics, and fitting a curve to them. Do this in the cell below. Your code should print out the value of $\alpha$ and be within 1% of the correct answer.

You may find the `scipy.signal.find_peaks` function useful.

In [ ]:
#Create function to be used for optimisation
def model_func(x,a):
    return x**a 

In [ ]:
from scipy import optimize

#fourier transform
ck = np.fft.rfft(ysqu)

#find the peaks
peaks, _ = signal.find_peaks(abs(ck))
wn = (2 * np.pi / np.max(tsqu)) * np.arange(ck.size)

#using equation given calculate the x and y data
ydata = abs(ck[peaks[1:]])/abs(ck[peaks[0]])
xdata = wn[peaks[1:]]

#optimise to find parameter alpha
popt,pcov = optimize.curve_fit(model_func,xdata,ydata)
print(popt[0])


## Task 2: 5 pts

Plot the amplitude spectrum of the square wave signal vs. wavenumber, $\omega$. This should show the harmonics occuring at odd values of $\omega$, i.e., 1, 3, 5, 7... Include on the plot the locations of the harmonics, denoted by circles. Plot a curve that fits $\omega$ and amplitude of the harmonics. Plot this in log-log to better illustrate the relation between $\omega$ and amplitude. Include all appropriate labels, legends, etc.

In [ ]:
#plot statements
plt.plot(wn,abs(ck),label="$c_{k}$")
plt.plot(peaks/10,abs(ck[peaks]),"o",label = "harmonic positions")
plt.plot(wn[wn>=1],abs(ck[peaks[0]]) *model_func(wn[wn>=1],popt),label = "Fitted curve")

plt.xlabel("$Wave number, \omega$")
plt.ylabel("Amplitude spectrum")
plt.legend()
plt.show()

In [ ]:
#Decided to plot two different graphs as wasnt sure which one was wanted
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

ax1.loglog(wn[peaks],abs(ck[peaks[0]])*model_func(wn[peaks],popt))
ax1.set_title('Without wavenumber amplitudes')
ax1.xaxis.set_label_text('$Wave number, \omega$')
ax1.yaxis.set_label_text('Amplitude')

ax2.loglog(wn,abs(ck),label="$c_{k}$")
ax2.loglog(wn[peaks],abs(ck[peaks[0]])*model_func(wn[peaks],popt))
ax2.set_title('With wavenumber amplitudes')
ax2.xaxis.set_label_text('$Wave number, \omega$')
ax2.yaxis.set_label_text('Amplitude')

plt.show()

## Problem 3: the solar cycle (25 pts)

The number of sunspots visible on the Sun is known to have cyclic behavior. The data file, "sunspots.txt" (included with the checkpoint), contains counts of the number of sunspots per month since 1749. The data contains two columns:
1. the time in years denoting the mid-point of the month
2. the number of sunspots observed in that month

## Task 1: 10 pts

In the cell below, write a code to compute the period of the primary mode of the solar cycle (i.e., the period corresponding to the highest peak in the amplitude spectrum, excluding the peak at $k=0$). Your code should print out the value of the period in years to within 0.2 years of the correct answer (as computed with this method).

In [ ]:
#extract data from file
time = np.loadtxt("sunspots.txt",usecols = 0)
sun_num = np.loadtxt("sunspots.txt",usecols = 1)

#create freq series and fourier arrays
freq = np.fft.rfftfreq(sun_num.size, 1/12)
ak = np.fft.rfft(sun_num)

#find the maximum above a certain value
maximum = np.max(abs(ak[0.05<freq]))
peak = freq[abs(ak)==maximum]

#calculate the peak
period = 1/peak[0]
print(period)

## Task 2: 10 pts

Using the period calculated previously, predict the time of the next solar maximum (i.e., the next time in the future when the number of sunspots will peak). Do this by creating a filter to include only the 30 closest values on each side of the peak. Create a new signal from the filtered Fourier coefficients and locate the last maximum. Use this to predict the time of the next maximum. Your code should print the month and year of the next maximum. Your answer should be accurate to within 2 months of the correct answer (as computed by this method).

Note, this method does not correctly calculate the time of the last peak, so your prediction will differ with the real prediction.

In [ ]:
#find the indice of the peak
find = np.nonzero(freq == peak)
indice = find[0][0]

#set everything above 30 and below 25 (30 is below 0) to 0 on either side of peak
ak[:indice-25] = 0
ak[indice+31:] = 0

#inverse fourier transform
ynew = np.fft.irfft(ak)

#find the last peak
peaks, _ = signal.find_peaks(ynew)
x = peaks[-1]
x2 = ynew[x]
time_peaks = time[ynew == x2]

#calculate the time of the next peak
time_passed = time[-1] - time_peaks
time_to_go = period - time_passed
next_peak = time[-1] + time_to_go

#print statements convert to months and years
year = np.floor(next_peak)
month = (next_peak[0] - year[0]) * 12
print("Full output: " + str(next_peak[0]))
print("Year: " +str(int(year[0])))
print("Month: " +str(round(month,6))+ " or March")


## Task 3: 5 pts

Make a plot including the original data of the number of sunspots vs. time, your filtered signal from the previous task, and a vertical line denoting the predicted time of the next maximum. Include all appropriate labels, units, and legends.

In [ ]:
plt.plot(time,sun_num,label ="Original data", color="orange")
plt.plot(time,ynew,label = "Filtered data",color ="r")
plt.axvline(x=next_peak[0],label = "Next maximum",color = "black")

plt.xlabel("Time (years)")
plt.ylabel("Number of sunspots")
plt.legend()
plt.show()